In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pycasso
import numpy as np

from Simulate import Simulate

# for visualizing result
import pandas as pd

In [3]:
# define solvers
lstsq=lambda x,y:np.linalg.lstsq(x,y,rcond=-1)[0]

def lasso(x,y):
    sol = pycasso.Solver(x,y)
    sol.train()
    return sol.coef()["beta"][-1]

# helper function for lasso solver
mse=lambda a,b:np.power(b-a,2).mean()

In [22]:
# helper function for presenting the correlation tables

def present(d):
    for k in d.keys():
        display(k)
        display(pd.DataFrame(d[k],index=["pearson corr","p-val"]))
        

# simple mediation, with uncorrelated OTU 

## solving with linear regression

In [23]:
# for the sake of comparison, set random seed
np.random.seed(0)

# variables
n=100
p_otu=4
p_met=10
b1=np.zeros((p_otu,1))
b2=np.zeros((p_otu,p_met))
b3=np.zeros((p_met,1))

b1[0,-1]=-2
b2[0,-1]=2
b3[0,-1]=-1.5

var=[n,b1,b2,b3]


# list of sample sizes to simulate
n_lst=[100,200,500,1000,2000]

# initialize for storing results
MSE_tb=pd.DataFrame(columns=n_lst)


for n in n_lst:
    var[0]=n
    
    # simulation and evaluation process begins
    s=Simulate(*var)
    s.simulate_mediation(np.random.rand,np.random.rand)
    s.estimate(lstsq)
    s.score()
    # simulation and evaluation process finishes
    
    MSE_tb[n]=s.MSE.values()
    
    
# formatting the results
MSE_tb.index=s.MSE.keys()
MSE_tb=MSE_tb.add_prefix("n = ")

idx=[]
for a in s.A_true.keys():
    temp=s.A_true[a]
    idx.append(a+" ({})".format(temp[temp!=0][0]))
    
MSE_tb.index=idx


display("MSE values of each coefficient vectors")
display(MSE_tb)

'MSE values of each coefficient vectors'

,n = 100,n = 200,n = 500,n = 1000,n = 2000
a11 (-2.0),0.049095,0.197633,0.148262,0.056369,0.005201
a21 (2.0),0.131021,0.072199,0.028333,0.013739,0.008912
a31 (-2.0),0.116975,0.225434,0.024573,0.017401,0.002600
a32 (-1.5),0.006452,0.008987,0.002078,0.000955,0.000879


In [24]:
display("pairwise correlation for n = {}".format(n))
present(s.pearsonCorr())

'pairwise correlation for n = 2000'

'exposure'

,"(0, 1)","(0, 2)","(0, 3)","(1, 0)","(1, 2)","(1, 3)","(2, 0)","(2, 1)","(2, 3)","(3, 0)","(3, 1)","(3, 2)"
pearson corr,0.026592,0.011189,0.042150,0.026592,0.039503,-0.008960,0.011189,0.039503,0.009464,0.042150,-0.008960,0.009464
p-val,0.234564,0.617013,0.059475,0.234564,0.077357,0.688818,0.617013,0.077357,0.672304,0.059475,0.688818,0.672304


'mediator'

,"(0, 1)","(0, 2)","(0, 3)","(0, 4)","(0, 5)","(0, 6)","(0, 7)","(0, 8)","(0, 9)","(1, 0)",...,"(8, 9)","(9, 0)","(9, 1)","(9, 2)","(9, 3)","(9, 4)","(9, 5)","(9, 6)","(9, 7)","(9, 8)"
pearson corr,-0.016119,-0.069802,-0.029478,0.004697,0.001699,-0.012569,0.015788,-0.011435,-0.003394,-0.016119,...,-0.002514,-0.003394,-0.046878,-0.023971,-0.016456,-0.024085,-0.01107,-0.010501,-0.065147,-0.002514
p-val,0.471249,0.001787,0.187582,0.833734,0.939467,0.574261,0.480398,0.609283,0.879433,0.471249,...,0.910535,0.879433,0.036057,0.283942,0.462026,0.281663,0.62077,0.638832,0.003560,0.910535


## solving with Lasso regression

Pycasso doesn't seem to be able to solve for cases when dependent variable $Y$ is a matrix, thus $a_{21}$ is skipped

In [7]:
# for the sake of comparison, set random seed
np.random.seed(0)

# variables
n=100
p_otu=4
p_met=10
b1=np.zeros((p_otu,1))
b2=np.zeros((p_otu,p_met))
b3=np.zeros((p_met,1))

b1[0,-1]=-2
b2[0,-1]=2
b3[0,-1]=-1.5

var=[n,b1,b2,b3]
keys=["a11","a31","a32"]


# list of sample sizes to simulate
n_lst=[100,200,500,1000,2000]

# initialize for storing results
MSE_tb=pd.DataFrame(columns=n_lst)


for n in n_lst:
    var[0]=n
    
    # simulation and evaluation process begins
    s=Simulate(*var)
    s.simulate_mediation(np.random.rand,np.random.rand)
    s.estimate(lasso)
    
    MSE=dict()
    for a in ["a11","a31","a32"]:
        m=mse(s.A[a],s._truth()[a])
        MSE[a]=m
    # simulation and evaluation process finishes
    
    MSE_tb[n]=MSE.values()
    
    
# formatting the results
MSE_tb.index=keys
MSE_tb=MSE_tb.add_prefix("n = ")

idx=[]
for a in keys:
    temp=s._truth()[a]
    idx.append(a+" ({})".format(temp[temp!=0][0]))
    
MSE_tb.index=idx


display("MSE values of each coefficient vectors")
display(MSE_tb)

Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.


'MSE values of each coefficient vectors'

,n = 100,n = 200,n = 500,n = 1000,n = 2000
a11 (-2.0),1.010275,1.019655,1.377836,1.170658,1.135729
a31 (-2.0),0.941830,0.950815,0.937616,0.948877,0.939833
a32 (-1.5),0.403925,0.386978,0.367895,0.392473,0.394886


In [8]:
display("pairwise correlation for n = {}".format(n))
present(s.pearsonCorr())

'pairwise correlation for n = 2000'

'exposure'

,pearson corr,p-val
"(0, 1)",0.026592,0.234564
"(0, 2)",0.011189,0.617013
"(0, 3)",0.042150,0.059475
"(1, 0)",0.026592,0.234564
"(1, 2)",0.039503,0.077357
"(1, 3)",-0.008960,0.688818
"(2, 0)",0.011189,0.617013
"(2, 1)",0.039503,0.077357
"(2, 3)",0.009464,0.672304
"(3, 0)",0.042150,0.059475


'mediator'

,pearson corr,p-val
"(0, 1)",-0.016119,0.471249
"(0, 2)",-0.069802,0.001787
"(0, 3)",-0.029478,0.187582
"(0, 4)",0.004697,0.833734
"(0, 5)",0.001699,0.939467
...,...,...
"(9, 4)",-0.024085,0.281663
"(9, 5)",-0.011070,0.620770
"(9, 6)",-0.010501,0.638832
"(9, 7)",-0.065147,0.003560


# simple mediation, with correlated OTU

In [9]:
from Random import TreeSimilarity

## solving with linear regression

In [16]:
# for the sake of comparison, set random seed
np.random.seed(0)

# variables
n=100
p_otu=4
p_met=10
b1=np.zeros((p_otu,1))
b2=np.zeros((p_otu,p_met))
b3=np.zeros((p_met,1))

b1[0,-1]=-2
b2[0,-1]=2
b3[0,-1]=-1.5

var=[n,b1,b2,b3]


# list of sample sizes to simulate
n_lst=[100,200,500,1000,2000]

# initialize for storing results
MSE_tb=pd.DataFrame(columns=n_lst)


for n in n_lst:
    var[0]=n
    
    # simulation and evaluation process begins
    s=Simulate(*var)
    t_exposure=TreeSimilarity(p_otu)
    t_mediator=TreeSimilarity(p_met)
    s.simulate_mediation(t_exposure.rand,t_mediator.rand)
    s.estimate(lstsq)
    s.score()
    # simulation and evaluation process finishes
    
    MSE_tb[n]=s.MSE.values()
    
    
# formatting the results
MSE_tb.index=s.MSE.keys()
MSE_tb=MSE_tb.add_prefix("n = ")

idx=[]
for a in s.A_true.keys():
    temp=s.A_true[a]
    idx.append(a+" ({})".format(temp[temp!=0][0]))
    
MSE_tb.index=idx


display("MSE values of each coefficient vectors")
display(MSE_tb)

'MSE values of each coefficient vectors'

,n = 100,n = 200,n = 500,n = 1000,n = 2000
a11 (-2.0),4.017746e+13,5.465892e+12,4.348811e+12,1.067416e+12,1.005369e+14
a21 (2.0),2.474821e+15,7.743143e+13,1.484928e+14,4.766246e+13,3.192789e+13
a31 (-2.0),5.791350e+13,1.175743e+13,2.663512e+14,4.340638e+10,2.507745e+14
a32 (-1.5),9.576666e-03,4.866871e-03,2.777468e-03,9.319753e-04,5.242444e-04


In [17]:
print("exposure")
t_exposure.draw()

print("mediator")
t_mediator.draw()

exposure
  ________________________________________________________________________ otu1
_|
 |                            ____________________________________________ otu2
 |___________________________|
                             |                                           _ otu3
                             |__________________________________________|
                                                                        |_ otu0

mediator
                 _______________________________ otu6
  ______________|
 |              |_______________________________ otu2
 |
 |                                                        ______________ otu8
 |                                        _______________|
_|                                       |               |______________ otu7
 |                          _____________|
 |                         |             |         _____________________ otu4
 |                         |             |________|
 |                         |        

In [18]:
display("pairwise correlation for n = {}".format(n))
present(s.pearsonCorr())

'pairwise correlation for n = 2000'

'exposure'

,pearson corr,p-val
"(0, 1)",-0.967078,0.000000e+00
"(0, 2)",-0.108170,1.242757e-06
"(0, 3)",0.946632,0.000000e+00
"(1, 0)",-0.967078,0.000000e+00
"(1, 2)",0.100931,6.111707e-06
"(1, 3)",-0.957889,0.000000e+00
"(2, 0)",-0.108170,1.242757e-06
"(2, 1)",0.100931,6.111707e-06
"(2, 3)",0.174225,4.267830e-15
"(3, 0)",0.946632,0.000000e+00


'mediator'

,pearson corr,p-val
"(0, 1)",0.057355,0.010303
"(0, 2)",-0.042108,0.059727
"(0, 3)",-0.005878,0.792758
"(0, 4)",0.011953,0.593185
"(0, 5)",0.048418,0.030369
...,...,...
"(9, 4)",0.007658,0.732149
"(9, 5)",-0.046540,0.037420
"(9, 6)",-0.056245,0.011877
"(9, 7)",0.017439,0.435696


## solving with lasso regression

In [19]:
# for the sake of comparison, set random seed
np.random.seed(0)

# variables
n=100
p_otu=4
p_met=10
b1=np.zeros((p_otu,1))
b2=np.zeros((p_otu,p_met))
b3=np.zeros((p_met,1))

b1[0,-1]=-2
b2[0,-1]=2
b3[0,-1]=-1.5

var=[n,b1,b2,b3]
keys=["a11","a31","a32"]


# list of sample sizes to simulate
n_lst=[100,200,500,1000,2000]

# initialize for storing results
MSE_tb=pd.DataFrame(columns=n_lst)


for n in n_lst:
    var[0]=n
    
    # simulation and evaluation process begins
    s=Simulate(*var)
    t_exposure=TreeSimilarity(p_otu)
    t_mediator=TreeSimilarity(p_met)
    s.simulate_mediation(t_exposure.rand,t_mediator.rand)
    s.estimate(lasso)
    
    MSE=dict()
    for a in ["a11","a31","a32"]:
        m=mse(s.A[a],s._truth()[a])
        MSE[a]=m
    # simulation and evaluation process finishes
    
    MSE_tb[n]=MSE.values()
    
    
# formatting the results
MSE_tb.index=keys
MSE_tb=MSE_tb.add_prefix("n = ")

idx=[]
for a in keys:
    temp=s._truth()[a]
    idx.append(a+" ({})".format(temp[temp!=0][0]))
    
MSE_tb.index=idx


display("MSE values of each coefficient vectors")
display(MSE_tb)

Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.
Training is over.


'MSE values of each coefficient vectors'

,n = 100,n = 200,n = 500,n = 1000,n = 2000
a11 (-2.0),2.125470,1.512799,1.415247,1.533631,1.485861
a31 (-2.0),0.986887,1.192656,1.277014,1.134060,1.141854
a32 (-1.5),0.382159,0.381392,0.371380,0.404649,0.382832


In [20]:
print("exposure")
t_exposure.draw()

print("mediator")
t_mediator.draw()

exposure
  ________________________________________________________________________ otu1
_|
 |                            ____________________________________________ otu2
 |___________________________|
                             |                                           _ otu3
                             |__________________________________________|
                                                                        |_ otu0

mediator
                 _______________________________ otu6
  ______________|
 |              |_______________________________ otu2
 |
 |                                                        ______________ otu8
 |                                        _______________|
_|                                       |               |______________ otu7
 |                          _____________|
 |                         |             |         _____________________ otu4
 |                         |             |________|
 |                         |        

In [21]:
display("pairwise correlation for n = {}".format(n))
present(s.pearsonCorr())

'pairwise correlation for n = 2000'

'exposure'

,pearson corr,p-val
"(0, 1)",-0.967078,0.000000e+00
"(0, 2)",-0.108170,1.242757e-06
"(0, 3)",0.946632,0.000000e+00
"(1, 0)",-0.967078,0.000000e+00
"(1, 2)",0.100931,6.111707e-06
"(1, 3)",-0.957889,0.000000e+00
"(2, 0)",-0.108170,1.242757e-06
"(2, 1)",0.100931,6.111707e-06
"(2, 3)",0.174225,4.267830e-15
"(3, 0)",0.946632,0.000000e+00


'mediator'

,pearson corr,p-val
"(0, 1)",0.057355,0.010303
"(0, 2)",-0.042108,0.059727
"(0, 3)",-0.005878,0.792758
"(0, 4)",0.011953,0.593185
"(0, 5)",0.048418,0.030369
...,...,...
"(9, 4)",0.007658,0.732149
"(9, 5)",-0.046540,0.037420
"(9, 6)",-0.056245,0.011877
"(9, 7)",0.017439,0.435696
